# Rendering Images through Gas Flow volume

In [ ]:
from jax import numpy as jnp

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True) 

import os
import sys

sys.path.append('../')

from rtnf.helpers import makedir, save_to_dir_np
from rtnf.truefield import generate_emission, init_rays, plot_2d, plot_3d
from rtnf.render import renderfn
from rtnf.network import Grid2, get_X

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'

In [ ]:
# resolution for generating IoR Field
res_eta = 64
eta_shape = (res_eta, res_eta, res_eta) 
# resolution for grid of light sources
res_lum = 64 
lum_shape = (res_lum, res_lum, res_lum)
# resolution (number of rays) for ray tracing
res = 64
# FOV angle for sensor (determines distance from scene)
fov_angle = 0.1

X_eta = get_X(res_eta)
X_lum = get_X(res_lum)

seed_ls = 0 #Seed for light source generation

save = False #Whether or not to save output

save_dir = '../exp/gasflow/' #Directory for saving truefield output

### Sample an IoR Field from a Gaussian Process

In [ ]:
grid_out = jnp.load('../data/fuel_injection_64.npy')
eta_Grid = Grid2(grid_vals = grid_out, cval=1.0003)
predict_eta = eta_Grid.interp5

In [ ]:
%matplotlib widget

# Visualize
plot_3d(grid_out, res_eta)

### Generate a dense light source field with Poisson Disk Sampling

In [ ]:
# Generate Field
lum_field, predict_lum_grid, ls = generate_emission('poisson', seed_ls, X_lum, lum_shape)

In [ ]:
%matplotlib widget

# Visualize
plot_3d(lum_field, res_lum)

### Render a Refracted Image

In [ ]:
# Initial Conditions
rays = init_rays(fov_angle, res)
s_vals = jnp.linspace(0, 1.2, 3)
# Render the image
ray_trace, ray_lum = renderfn(rays, predict_eta, predict_lum_grid, s_vals)

In [ ]:
%matplotlib inline

# Plot the image
plot_2d(ray_lum.sum(1).reshape(res, res)[::-1, ::-1].T)

### Save Output for Training

In [ ]:
if save: 
    makedir(save_dir)
    save_to_dir_np(save_dir, X_eta, rays, s_vals, lum_field, ray_lum, grid_out)